# Sql in Python
___
Author: Kamil Pazik

email: pazik.kamil@gmail.com

phone: +48 721 114 737

## TOC
1. [Refresher of OOP](#Refresher-oop)
2. [Ways of connecting Python with databases](#Ways-of-connecting-Python-with-databases)
3. [Installing prerequisites](#Installing-prerequisites)
4. [Orm](#Orm)
5. [Data models](#Data-models)
  1. [Exercises](#Data-models-exercises)
7. [Quering and data manipulations](#Quering-and-data-manipulations)
8. [Simple exploratory data analysis](#Simple-exploratory-data-analysis)

### Refresher-oop

* What is class ? why we need oop ?
* What is instance/object ?
* What are methods / attributes / properties ?
* What are staticmethod / classmethod ?

### Ways of connecting Python with databases
* Directly using specific libs [sqlite](https://docs.python.org/3/library/sqlite3.html), [psycopg2](http://initd.org/psycopg/docs/)
* Using ORM like [sqllachemy](https://www.sqlalchemy.org/)
* Using pandas 

### Installing prerequisites

* Database tool - [Dbeaver](https://dbeaver.io/)
* Sqlalchemy with pyscopg2
  ```bash
  conda install sqlalchemy
  conda install psycopg2
  ```
* Installation of postgres (your linux machine)
  ```bash
  sudo apt-get update
  sudo apt-get install postgresql postgresql-contrib
  ```
  after installation
  ```bash
  su - postgres
  psql -d template1 -c "ALTER USER postgres WITH PASSWORD 'postgres';"
  ```

### Orm
* Object relational mapping,
* Maps data between systems (Python classes to database tables)

### Data models